In [7]:
import streamlit as st
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,accuracy_score
from sklearn.tree import DecisionTreeClassifier


In [8]:
#nltk.download()

In [9]:
# Load the dataset
df = pd.read_csv('C:/Users/LENOVO/OneDrive/Desktop/project/sentimental/sentimental/sentiment_dataset_50k.csv')

In [10]:
df

,text,sentiment
0,The printer did its job as expected.,neutral
1,This laptop is fantastic and exceeded my expec...,positive
2,Extremely dissatisfied with this headphone.,negative
3,The mouse worked perfectly and made my day!,positive
4,It's an average headphone. Just fine.,neutral
...,...,...
49995,The mouse worked perfectly and made my day!,positive
49996,The keyboard is neither good nor bad.,neutral
49997,Extremely dissatisfied with this tablet.,negative
49998,This printer is fantastic and exceeded my expe...,positive


In [11]:
import re
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Define preprocessing function
def preprocess(text):
    text = text.lower()
    tokens = re.findall(r'\b[a-z]+\b', text)  # only alphabetic words
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

df['clean_text'] = df['text'].apply(preprocess)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_text'])  #  Now this column exists
y = df['sentiment']


In [13]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Train a classifier
model = DecisionTreeClassifier(max_depth=9).fit(X_train, y_train)


In [15]:
# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy score:",accuracy_score(y_test, y_pred)*100,("%"))


              precision    recall  f1-score   support

    negative       1.00      0.80      0.89      2922
     neutral       1.00      1.00      1.00      3038
    positive       0.88      1.00      0.93      4040

    accuracy                           0.94     10000
   macro avg       0.96      0.93      0.94     10000
weighted avg       0.95      0.94      0.94     10000

Accuracy score: 94.24 %


In [16]:
# Predict sentiment of new sentence
def predict_sentiment(sentence):
    processed = preprocess(sentence)
    vectorized = vectorizer.transform([processed])
    return model.predict(vectorized)[0]


In [17]:
#user input
new_sentence=input("Enter a sentence to predict sentiment: ")
predicted_sentiment = predict_sentiment(new_sentence)
print(f"The predicted sentiment for the sentence is: {predicted_sentiment}")

The predicted sentiment for the sentence is: positive


In [18]:
import pickle

In [19]:
# Save model
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save vectorizer
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)
